Setup

In [4]:

!pip -q install --upgrade diffusers==0.30.2 transformers accelerate safetensors opencv-python einops
import os, cv2, numpy as np, torch
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8 else torch.float16


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 93.9 MB/s eta 0:00:00


Mounting gDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading Upscale model

In [ ]:
up = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    torch_dtype=dtype,
).to(device)

try:
    up.enable_xformers_memory_efficient_attention(False)
    up.unet.set_default_attn_processor()
except Exception:
    pass

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Setting up and running batching super res

In [ ]:

# === Paths ===
inputFolder  = "/content/drive/MyDrive/Upsampling/extracted/"      # folder with low-res images
outputFolder = "/content/drive/MyDrive/Upsampling/output/StableDiffusion/v1"
os.makedirs(outputFolder, exist_ok=True)

# === Params ===
steps = 20
cfg   = 0.0     # fidelity
maxInput = 512  # for downscaling larger input imgs before x4 upscaling

prompt = ""     # empty for now since no need for generating new content
negative = "painting, cartoon, oversharpened, mosaic, artifacts"

imageExtensions = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp"}

def prepare_lr(image: Image.Image, maxSide=512):
    w, h = image.size
    scale = min(maxSide / max(w, h), 1.0)
    if scale < 1.0:
        new_w, new_h = int(w*scale), int(h*scale)
        image = image.resize((new_w, new_h), Image.BICUBIC)
    return image

def run_one(path):
    name = os.path.splitext(os.path.basename(path))[0]
    bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    if bgr is None:
        print("Skip (cannot read):", path); return
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(rgb)
    lr  = prepare_lr(pil, maxSide=maxInput)

    out = up(
        prompt=prompt,
        image=lr,
        num_inference_steps=steps,
        guidance_scale=cfg,
    ).images[0]

    outBGR = cv2.cvtColor(np.array(out), cv2.COLOR_RGB2BGR)
    savePath = os.path.join(outputFolder, f"{name}_x4.png")
    cv2.imwrite(savePath, outBGR)
    print("Saved:", savePath)

files = [fn for fn in sorted(os.listdir(inputFolder)) if os.path.splitext(fn.lower())[1] in imageExtensions]
for fn in files:
    run_one(os.path.join(inputFolder, fn))


  0%|          | 0/20 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/Upsampling/output/StableDiffusion/v1/scene004_selection_x4.png


  0%|          | 0/20 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/Upsampling/output/StableDiffusion/v1/scene004_selection2_x4.png


  0%|          | 0/20 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/Upsampling/output/StableDiffusion/v1/scene004_selection_zoom_2_x4.png


  0%|          | 0/20 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/Upsampling/output/StableDiffusion/v1/scene004_selection_zoomed_x4.png
